In [1]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the .env file
api_key = os.getenv('GEMINI_API_KEY')

# Initialize the Google AI client
client = genai.Client(api_key=api_key)

# Generate content
response = client.models.generate_content(
    model='gemini-2.0-flash-exp', contents='What is your name?'
)
print(response.text)




I am a large language model, trained by Google.



In [2]:
from google.genai.types import GenerateContentConfig, Tool
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from IPython.display import display, HTML, Markdown
import io
import json
import re

In [3]:
MODEL = "gemini-2.0-flash-exp"

In [4]:

COMPANY = 'Proximus'

In [5]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates and metrics to write research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

Add to the report a link to the most recent company logo, a summary of companies who offer services to them, a comprehensive summary of the key and their roles in the company, a summary of departments of the company. 

Feel free to plan your work and talk about it, but when you start writing the report,
put a line of dashes (---) to demarcate the report itself, and say nothing else after
the report has finished.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0)
response_stream = client.models.generate_content_stream(
    model=MODEL, config=config, contents=[COMPANY])

report = io.StringIO()
for chunk in response_stream:
  candidate = chunk.candidates[0]

  for part in candidate.content.parts:
    if part.text:
      display(Markdown(part.text))

      # Find and save the report itself.
      if m := re.search('(^|\n)-+\n(.*)$', part.text, re.M):
          # Find the starting '---' line and start saving.
          report.write(m.group(2))
      elif report.tell():
        # If there's already something in the buffer, keep recording.
        report.write(part.text)

    else:
      print(json.dumps(part.model_dump(exclude_none=True), indent=2))

  # You must enable Google Search Suggestions
  if gm := candidate.grounding_metadata:
    if sep := gm.search_entry_point:
      display(HTML(sep.rendered_content))

Okay

, I will create a company report for Proximus. Here's my

 plan:

1.  **Company Overview:** I'll start by gathering

 general information about Proximus, such as its industry, headquarters, and a brief history.
2.  **Logo:** I'll find the most

 recent company logo and include a link to it.
3.  **Services:** I'll research what services Proximus offers and identify companies that provide

...

 Coune, Karel De Gucht, Béatrice de Mahieu, Martin De Prycker, Audrey Hanard, Ibrahim Ouassari, Catherine Rutten, Joachim Sonne.

**Departments:**

The Proximus Group is structured into several departments:

*   CEO's Department
*   Consumer Market


*   Enterprise Market
*   Corporate Affairs
*   Network & Wholesale
*   Customer Operations
*   Human Capital
*   Digital & IT
*   Finance

**Recent News/Updates:**

*   **Proximus Global:** Proximus has created Proximus Global, integrating its international

 activities under one umbrella, including BICS, Telesign, and Route Mobile.
*   **Fiber Network:** Proximus' fiber network has received recognition for its speed and performance.
*   **Scarlet Offer:** Scarlet has launched a new offer with increased data volumes at the same price.
*   

**Supply Chain:** Proximus is testing cargo sailboats in its supply chain.
*   **Data Center Sale:** Proximus sold its data center business to Datacenter United.
*   **Luxembourg Towers Sale:** Proximus sold its Luxembourg towers to InfraRed Capital Partners.

Proximus

 is actively expanding its fiber network in Belgium, aiming for 95% coverage by 2032. The company is also focusing on digital transformation and international growth.
---


In [12]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates and metrics to write research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

Add to the report a link to the most recent company logo, the company colors in rgb, a summary of companies who offer services to them, a comprehensive summary of the key employees and their roles in the company, a summary of departments of the company. 

Feel free to plan your work and talk about it, but when you start writing the report,
put a line of dashes (---) to demarcate the report itself, and say nothing else after
the report has finished.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_k=1, top_p=1)
response = client.models.generate_content(
    model=MODEL, config=config, contents=[COMPANY])

print(response.text)


Okay, I will create a company report for Proximus. Here's my plan:

1.  **Company Overview:** I'll start by gathering general information about Proximus, such as its industry, headquarters location, and a brief history.
2.  **Logo and Colors:** I'll find the most recent company logo and identify its official colors in RGB format.
3.  **Service Providers:** I'll research companies that provide services to Proximus.
4.  **Key Employees:** I'll identify key employees and their roles within the company.
5.  **Company Departments:** I'll outline the main departments within Proximus.
6.  **Recent News and Updates:** I'll look for any recent news or updates about the company.

Now, let's start with the research.
Okay, I have gathered a lot of information about Proximus. Here's the company report:

---
**Company Report: Proximus**

**1. Company Overview:**

Proximus Group is a Belgian telecommunications and digital services provider. It is the incumbent telecom operator in Belgium, offering a 

**Proximus Key Employees and Roles**

In [26]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates, metrics, linkedin pages, common news articles, financial reports and public records to write a research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

The report should only contain a comprehensive summary of the employees in the company. 

The report should contain, the complete first name and family name, the role that they have within the company and the department that they make part of.

Do not include abbreviations in names, and exclude employees whose full first and last names cannot be found.

Give me as much grounded names that you can find with your research and put them all in the report.

The report should be written in this style example:

Use this JSON schema:

Proximus_employees = {'first_name':str, 'family_name':str, 'role':str, 'department':str}
Return: list[Proximus_employees]

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_p=0.9)
response = client.models.generate_content(
    model=MODEL, config=config, contents=[COMPANY])

print(response.text)


```json
[
    {
        "first_name": "Guillaume",
        "family_name": "Boutin",
        "role": "Chief Executive Officer",
        "department": "CEO's department"
    },
    {
        "first_name": "Stefaan",
        "family_name": "De Clerck",
        "role": "Chairman of the Board",
        "department": "Board of Directors"
    },
     {
        "first_name": "Ben",
        "family_name": "Appel",
        "role": "Corporate Affairs Lead",
        "department": "Corporate Affairs"
    },
    {
        "first_name": "Jim",
        "family_name": "Casteele",
        "role": "Consumer Market Lead",
         "department": "Consumer Market"
    },
    {
        "first_name": "Anne-Sophie",
        "family_name": "Lotgering",
        "role": "Enterprise IT Services & Segments Lead",
        "department": "Enterprise Market"
    },
    {
        "first_name": "Antonietta",
        "family_name": "Mastroianni",
        "role": "Digital & IT Lead",
        "department":"Digital & IT"
   